Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from dataclasses import dataclass
import random
from icecream import ic
from tqdm.auto import tqdm

logging.basicConfig(level=logging.DEBUG)

In [ ]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2): ##It create all possible combinations of cities and with geodesic i calculate all distances
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)).km
CITIES.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [ ]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy Algorithm

In [ ]:
def change_position(tsp):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(tsp) - 2):
            for j in range(i + 1, len(tsp) - 1):
                # Check if the inversion would reduce the path length
                if DIST_MATRIX[tsp[i - 1], tsp[i]] + DIST_MATRIX[tsp[j], tsp[j + 1]] > DIST_MATRIX[tsp[i - 1], tsp[j]] + DIST_MATRIX[tsp[i], tsp[j + 1]]:
                    tsp[i], tsp[j] = tsp[j], tsp[i]
                    improved = True
    return tsp

In [ ]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf  # set all distances from current city to infinity, in order to exclude in the next step
    closest = np.argmin(dist[city])
    logging.debug(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
    visited[closest] = True
    city = closest
    tsp.append(int(city))

logging.debug(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
tsp.append(tsp[0])

logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

tsp = change_position(tsp)

logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")


## Second Algorithm: EA

In [ ]:
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

def fitness(i : Individual):
    genome = i.genome
    assert genome[0] == genome[-1]
    assert set(genome) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(genome, genome[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
        
    return tot_cost

def parent_selection(population, tournament_size=3):
    candidates = random.sample(population, tournament_size)
    
    candidates.sort(key=lambda x: x.fitness, reverse=True)
    
    for candidate in candidates:
        if random.random() < 0.8:
            return candidate
    return candidates[0]

def insert_muatation(p: Individual):
    genome = p.genome.copy()
    
    # Select two different positions, avoiding the first and last element (0 and 0) of the path
    pos1, pos2 = random.sample(range(1, len(genome) - 1), 2)
    
    gene = genome[pos1]
    
    # Remove the element from the original position and insert it in the new position
    genome = np.delete(genome, pos1)
    genome = np.insert(genome, pos2, gene)
   
    return Individual(genome=np.array(genome), fitness=None)

def inversion_mutation(individual: Individual) -> Individual:
    genome = individual.genome.copy()

    pos1, pos2 = sorted(random.sample(range(1, len(genome) - 1), 2))

    genome[pos1:pos2+1] = genome[pos1:pos2+1][::-1]

    return Individual(genome=genome, fitness=None)

   
def crossover(p1: Individual, p2: Individual):
    genome1 = p1.genome[1:-1]  
    genome2 = p2.genome[1:-1]
    size = len(genome1)
    
    # Select a random range
    start, end = sorted(random.sample(range(size), 2))
    
    # Copy the segment from the first parent
    child_genome = [-1] * size
    child_genome[start:end] = genome1[start:end]
    
    # Fill with the segment from the second parent
    for city in genome2:
        if city not in child_genome:
            for i in range(size):
                if child_genome[i] == -1:
                    child_genome[i] = city
                    break

    child_genome = np.array([0] + child_genome + [0])
    return Individual(genome=child_genome, fitness=None)



In [ ]:
POPULATION_SIZE = 20
OFFSPRING_SIZE = 4
MAX_GENERATIONS = 10_000

mutation = insert_muatation
num_cities = len(CITIES)

# Genarate population of random paths
population = [
    Individual(
        genome=np.array([0] + random.sample(range(1, num_cities), num_cities - 1) + [0]),
        fitness=None
    )   for _ in range(POPULATION_SIZE-1)]

# Insert the path of the prevous algorithm
population.insert(0, Individual(genome= tsp, fitness=None))

for i in population:
    i.fitness = fitness(i)

for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        if np.random.random() < 0.5:
            p = parent_selection(population)
            o = mutation(p)
        else:
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o = crossover(p1, p2)

        offspring.append(o)
    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=False)
    population = population[:POPULATION_SIZE]

logging.info(f"result EA: Found a path of {len(population[0].genome)-1} steps, total length {tsp_cost(population[0].genome):.2f}km")

population[0]. genome = change_position(population[0].genome)

logging.info(f"result: Found a path of {len(population[0].genome)-1} steps, total length {tsp_cost(population[0].genome):.2f}km")